In [24]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import random
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [2]:
train = pd.read_csv('train.csv')
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 
train['occyp_type'].loc[(train.occyp_type == 'NAN')&(train.DAYS_EMPLOYED > 0)]='Unemployed'

train.fillna('NAN', inplace=True) 
train['occyp_type'].loc[(train.occyp_type == 'NAN')&(train.DAYS_EMPLOYED < 0)]='Missing'

In [3]:
test = pd.read_csv('test.csv')
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)
test['occyp_type'].loc[(test.occyp_type == 'NAN')&(test.DAYS_EMPLOYED > 0)]='Unemployed'
test['occyp_type'].loc[(test.occyp_type == 'NAN')&(test.DAYS_EMPLOYED < 0)]='Missing'

submit = pd.read_csv('submission.csv')

In [4]:
object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

In [5]:
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

In [6]:
test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

In [7]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

In [9]:
random.seed(42)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    lgb = LGBMClassifier(n_estimators=1000)
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')

====================================1============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.652377	valid_1's multi_logloss: 0.752522
[200]	training's multi_logloss: 0.56559	valid_1's multi_logloss: 0.737156
[300]	training's multi_logloss: 0.501921	valid_1's multi_logloss: 0.731843
Early stopping, best iteration is:
[348]	training's multi_logloss: 0.475822	valid_1's multi_logloss: 0.729304


====================================2============================================
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.646476	valid_1's multi_logloss: 0.764955
[200]	training's multi_logloss: 0.560614	valid_1's multi_logloss: 0.751681
[300]	training's multi_logloss: 0.497532	valid_1's multi_logloss: 0.748007
Early stopping, best iteration is:
[281]	training's multi_logloss: 0.508566	valid_1's multi_logloss: 0.747717


====================================3====

In [17]:
xgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values
    params={
    'objective':'multi:softprob',
    'random_state':71,
    'n_estimators':1000
    }
    model = XGBClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric = 'mlogloss',early_stopping_rounds=30, verbose=100)
    xgb_models[fold]=model
    print(f'================================================================================\n\n')

====================================1============================================
[0]	validation_0-mlogloss:0.97436	validation_1-mlogloss:0.97758
[100]	validation_0-mlogloss:0.56747	validation_1-mlogloss:0.73381
[200]	validation_0-mlogloss:0.44779	validation_1-mlogloss:0.72234
[211]	validation_0-mlogloss:0.43764	validation_1-mlogloss:0.72251


====================================2============================================
[0]	validation_0-mlogloss:0.97304	validation_1-mlogloss:0.97925
[100]	validation_0-mlogloss:0.56072	validation_1-mlogloss:0.74882
[200]	validation_0-mlogloss:0.44335	validation_1-mlogloss:0.74027
[235]	validation_0-mlogloss:0.41306	validation_1-mlogloss:0.74266


====================================3============================================
[0]	validation_0-mlogloss:0.97442	validation_1-mlogloss:0.97858
[100]	validation_0-mlogloss:0.56273	validation_1-mlogloss:0.74339
[200]	validation_0-mlogloss:0.44721	validation_1-mlogloss:0.73763
[204]	validation_0-mlogloss:0.

In [26]:
rf_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values
    params={'n_estimators':1000,'random_state':71,'criterion':'gini','verbose':1,'class_weight':'balanced','n_jobs':-1,'oob_score':True}
    model=RandomForestClassifier(**params)
    model.fit(X_train, y_train)
    rf_models[fold]=model
    print(f'================================================================================\n\n')

====================================1============================================


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    7.0s finished




====================================2============================================


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    7.8s finished




====================================3============================================


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    9.2s finished




====================================4============================================


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   13.6s finished




====================================5============================================


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   16.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   22.0s finished


In [27]:
svc_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values
    params={'gamma':'auto','probability':True,'random_state':71,'class_weight':'balanced','kernel':'poly','shrinking':True,'verbose':True}
    svc_model=SVC(**params)
    model.fit(X_train, y_train)
    svc_models[fold]=model
    print(f'================================================================================\n\n')

====================================1============================================


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   23.2s finished




====================================2============================================


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   25.7s finished




====================================3============================================


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   19.7s finished




====================================4============================================


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   24.6s finished




====================================5============================================


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   24.9s finished


In [28]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += (lgb_models[fold].predict_proba(test)/20)
    submit.iloc[:,1:] += (xgb_models[fold].predict_proba(test)/20)
    submit.iloc[:,1:] += (rf_models[fold].predict_proba(test)/20)
    submit.iloc[:,1:] += (svc_models[fold].predict_proba(test)/20)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    2.6s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    3.4s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.7s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    2.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Paral

In [29]:
submit

,index,0,1,2
0,26457,0.046012,0.144719,0.809268
1,26458,0.225745,0.273219,0.501036
2,26459,0.045184,0.095037,0.859780
3,26460,0.087023,0.096783,0.816194
4,26461,0.091198,0.164841,0.743961
...,...,...,...,...
9995,36452,0.122803,0.297256,0.579941
9996,36453,0.207766,0.428965,0.363269
9997,36454,0.012381,0.052253,0.935366
9998,36455,0.305808,0.263339,0.430852


In [30]:
submit.to_csv('20210518_ensemble.csv', index=False) #0.7044283413 모델이 중요!